# Google Datebase Cloud SQL MSSQL Langchain Document Loader

[Google Database Cloud SQL for MSSQL](https://cloud.google.com/sql/MSSQL).

Save, load and delete langchain documents from `MSSQL` database.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-MSSQL-python/blob/main/docs/document_loader.ipynb)

## Pre-reqs

### Install package

In [ ]:
%pip install langchain-google-cloud-sql-mssql

In [1]:
from langchain_google_cloud_sql_mssql import MSSQLEngine, MSSQLLoader, MSSQLDocumentSaver
from langchain_core.documents import Document
import sqlalchemy

### Setup Cloud SQL MSSQL database and configure IAM user access to database

Setup Cloud SQL MSSQL database: https://cloud.google.com/sql/docs/MSSQL/create-instance

Login datebase using IAM database authentication: https://cloud.google.com/sql/docs/MSSQL/iam-logins

After confirmed access to database in the runtime environment of this notebook, filling the following values and run the cell before running the following example scripts.



In [3]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.

# Please fill in these values.
project_id = "langchain-cloud-sql-testing" #@param {type:"string"}

#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance.
region = "us-central1" #@param {type:"string"}
instance_id = "test-mssql-instance" #@param {type:"string"}
db_name = "test" #@param {type:"string"}
user_name = "sqlserver" #@param {type:"string"}
password = "Csql-admin-1234" #@param {type:"string"}

#@markdown Please specify a table name for demo purpose.
table_name = "test-default" #@param {type:"string"}

## Basic Usage

### Save documents in default table

Create a table of default schema via `MSSQLEngine.init_document_table(<table_name>)`. Table Columns:
- page_content (type: text)
- langchain_metadata (type: JSON)

In [4]:
engine = MSSQLEngine.from_instance(
    project_id=project_id, region=region, instance=instance_id, database=db_name, user=user_name, password=password
)
with engine.connect() as conn:
    conn.execute(sqlalchemy.text(f'DROP TABLE IF EXISTS "{table_name}";'))
    conn.commit()
engine.init_document_table(table_name)
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text(f"select * from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='{table_name}';"))
    for row in result.fetchall():
        print(row)

('test', 'dbo', 'test-default', 'page_content', 1, None, 'NO', 'nvarchar', -1, -1, None, None, None, None, None, None, 'UNICODE', None, None, 'SQL_Latin1_General_CP1_CI_AS', None, None, None)
('test', 'dbo', 'test-default', 'langchain_metadata', 2, None, 'YES', 'nvarchar', -1, -1, None, None, None, None, None, None, 'UNICODE', None, None, 'SQL_Latin1_General_CP1_CI_AS', None, None, None)


Save langchain documents with `MSSQLDocumentSaver.add_documents(<documents>)`.

In [5]:
test_docs = [
    Document(
        page_content="Apple Granny Smith 150 0.99 1",
        metadata={"fruit_id": 1},
    ),
    Document(
        page_content="Banana Cavendish 200 0.59 0",
        metadata={"fruit_id": 2},
    ),
    Document(
        page_content="Orange Navel 80 1.29 1",
        metadata={"fruit_id": 3},
    ),
]
saver = MSSQLDocumentSaver(engine=engine, table_name=table_name)
saver.add_documents(test_docs)

In [29]:
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text(f'select * from "{table_name}";'))
    print(result.keys())
    print(result.fetchall())

RMKeyView(['page_content', 'langchain_metadata'])
[('Apple Granny Smith 150 0.99 1', '{"fruit_id": 1}'), ('Banana Cavendish 200 0.59 0', '{"fruit_id": 2}'), ('Orange Navel 80 1.29 1', '{"fruit_id": 3}')]


### Load documents from default table

Load langchain document with `MSSQLLoader.load()` or `MSSQLLoader.lazy_load()`. `lazy_load` returns a generator that only queries database during the iteration.

In [6]:
loader = MSSQLLoader(engine=engine, table_name=table_name)
docs = loader.lazy_load()
for doc in docs:
    print("Loaded documents:", doc)

[('page_content', 99), ('langchain_metadata', 99)]


AttributeError: 'str' object has no attribute 'items'

### Load documents via query

Other than loading documents from a table, we can also choose to load documents from a view generated from a SQL query. For example:

In [ ]:
loader = MSSQLLoader(
    engine=engine,
    query=f"select * from `{table_name}` where JSON_EXTRACT(langchain_metadata, '$.fruit_id') = 1;"
)
onedoc = loader.load()
print("Loaded documents:", onedoc)

The view generated from SQL query can have different schema than default table. In such cases, the behavior of MSSQLLoader is the same as loading from table with non-default schema. Please refer to section [Load documents with customized document page content & metadata](#Load-documents-with-customized-document-page-content-&-metadata).

### Delete documents in default table

Delete a list of langchain documents from MSSQL table with `MSSQLDocumentSaver.delete(<documents>)`.

For table with default schema (page_content, langchain_metadata), the deletion criteria is:

A `row` should be deleted if there exists a `document` in the list, such that
- `document.page_content` equals `row[page_content]`
- `document.metadata` equals `row[langchain_metadata]`

In [ ]:
loader = MSSQLLoader(
    engine=engine,
    table_name=table_name
)
docs = loader.load()
print("Documents before delete:", docs)
saver.delete(onedoc)
print("Documents after delete:", loader.load())

## Advanced Usage

### Load documents with customized document page content & metadata

First we prepare an example table with non-default schema, and populate it with some arbitary data.

In [12]:
with engine.connect() as conn:
    conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS `{table_name}`"))
    conn.commit()
    conn.execute(
        sqlalchemy.text(
            f"""
            CREATE TABLE IF NOT EXISTS `{table_name}`(
                fruit_id INT AUTO_INCREMENT PRIMARY KEY,
                fruit_name VARCHAR(100) NOT NULL,
                variety VARCHAR(50),  
                quantity_in_stock INT NOT NULL,
                price_per_unit DECIMAL(6,2) NOT NULL,
                organic TINYINT(1) NOT NULL
            )
            """
        )
    )
    conn.execute(
        sqlalchemy.text(
            f"""
            INSERT INTO `{table_name}` (fruit_name, variety, quantity_in_stock, price_per_unit, organic)
            VALUES
                ('Apple', 'Granny Smith', 150, 0.99, 1),
                ('Banana', 'Cavendish', 200, 0.59, 0),
                ('Orange', 'Navel', 80, 1.29, 1);
            """
        )
    )
    conn.commit()

If we still load langchain document with default option of `MSSQLLoader` from this example table, the page_content of loaded documents will be the first column of the table, and metadata will be conists of key-value pairs of all the other columns.

In [ ]:
loader = MSSQLLoader(
    engine=engine,
    table_name=table_name,
)
print(loader.load())

We can specify the content and metadata we want to load by setting the `content_columns` and `metadata_columns` when initializing the `MSSQLLoader`.

For example here, the values of columns in `content_columns` will be joined together into a space-separated string, as `page_content` of loaded documents, and `metadata` of loaded documents will only contain key-value pairs of columns specified in `metadata_columns`.

In [ ]:
loader = MSSQLLoader(
    engine=engine,
    table_name=table_name,
    content_columns=[
        "variety",
        "quantity_in_stock",
        "price_per_unit",
        "organic",
    ],
    metadata_columns=["fruit_id", "fruit_name"],
)
print(loader.load())

### Save document into table with customized metadata

In order to save langchain document into table with customized metadata fields. We need first create such a table via `MSSQLEngine.init_document_table()`, and specify the list of `metadata_columns` we want it to have. In this example, the created table will have table columns:
- page_content (type: text)
- fruit_name (type text)
- organic (type tinyint(1))
- langchain_metadata (type: JSON)

`store_metadata` indicate if we want to store metadata fields other than the ones specified in `metadata_columns`, those extra metadata will be stored in `langchain_metadata` column. In case of `store_metadata=False`, `langchain_metadata` will not be created.

In [15]:
with engine.connect() as conn:
    conn.execute(sqlalchemy.text(f"DROP TABLE IF EXISTS `{table_name}`"))
    conn.commit()
engine.init_document_table(
    table_name, 
    metadata_columns=[
        sqlalchemy.Column(
            "fruit_name",
            sqlalchemy.UnicodeText,
            primary_key=False,
            nullable=True,
        ),
        sqlalchemy.Column(
            "organic",
            sqlalchemy.Boolean,
            primary_key=False,
            nullable=True,
        ),
    ],
    store_metadata=True
)

Save documents with `MSSQLDocumentSaver.add_documents(<documents>)`. As you can see in this example, 
- `document.page_content` will be saved into `page_content` column.
- `document.metadata.fruit_name` will be saved into `fruit_name` column.
- `document.metadata.organic` will be saved into `organic` column.
- `document.metadata.fruit_id` will be saved into `langchain_metadata` column in JSON format.

In [ ]:
test_docs = [
    Document(
        page_content="Granny Smith 150 0.99",
        metadata={"fruit_id": 1, "fruit_name": "Apple", "organic": 1},
    ),
]
saver = MSSQLDocumentSaver(engine=engine, table_name=table_name)
saver.add_documents(test_docs)
with engine.connect() as conn:
    result = conn.execute(sqlalchemy.text(f"select * from `{table_name}`;"))
    print(result.keys())
    print(result.fetchall())

### Delete documents in table with customized metadata

We can also delete documents from table with customized metadata columns via `MSSQLDocumentSaver.delete(<documents>)`. The deletion criteria is:

A `row` should be deleted if there exists a `document` in the list, such that
- `document.page_content` equals `row[page_content]`
- For every metadata field `k` in `document.metadata`
    - `document.metadata[k]` equals `row[k]` or `document.metadata[k]` equals `row[langchain_metadata][k]`
- There no extra metadata field presents in `row` but not in `document.metadata`.



In [ ]:
loader = MSSQLLoader(
    engine=engine,
    table_name=table_name
)
docs = loader.load()
print("Documents before delete:", docs)
saver.delete(docs)
print("Documents after delete:", loader.load())